# Cell Segmentation

Import required libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from skimage import io, filters, segmentation
from skimage.morphology import disk, closing
from skimage.feature import peak_local_max
from scipy import ndimage

Load and display the image

In [2]:
# Create a sample image for demonstration
img = np.zeros((100, 100))
rr, cc = np.ogrid[0:100, 0:100]
img[(rr-50)**2 + (cc-50)**2 < 900] = 1
plt.imshow(img, cmap='gray')
plt.axis('off')
plt.savefig('original.png')
plt.close()

Pre-process the image using gaussian smoothing and morphological closing

In [3]:
smoothed = filters.gaussian(img, sigma=1)
closed = closing(smoothed, disk(3))

Watershed segmentation using membrane intensity as elevation map

In [4]:
distance = ndimage.distance_transform_edt(-closed)
local_max_coords = peak_local_max(distance, min_distance=20)
markers = np.zeros_like(img)
markers[tuple(local_max_coords.T)] = 1
markers = ndimage.label(markers)[0]
labels = segmentation.watershed(-closed, markers, mask=img>0)

Visualize the segmentation result

In [5]:
plt.imshow(labels, cmap='nipy_spectral')
plt.axis('off')
plt.savefig('segmentation.png')
plt.close()